# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = '../WeatherPy/output_data/clean_cities.csv'
cities_df = pd.read_csv(csv_path)
cities_df.head()

,City Name,Latitude,Longitude,Daily High,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nikolskoye,59.70,30.79,60.80,87,90,4.47,RU,2020-07-10
1,Tiksi,71.69,128.87,44.71,81,59,8.97,RU,2020-07-10
2,Rikitea,-23.12,-134.97,68.00,64,64,16.46,PF,2020-07-10
3,Punta Arenas,-53.15,-70.92,26.60,92,75,6.93,CL,2020-07-10
4,Upernavik,72.79,-56.15,42.35,86,94,7.65,GL,2020-07-10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)
locations = cities_df[['Latitude','Longitude']].astype(float)


In [8]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=cities_df.Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.5)

fig.add_layer(heat_layer)



In [9]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows 
# returned by your API requests to a reasonable number.

finest_cities = cities_df.loc[
    (cities_df['Daily High'] < 75) &
    (cities_df['Daily High'] > 65) &
    (cities_df['Wind Speed'] < 10) &
    (cities_df['Cloudiness'] < 2) &
    (cities_df['Humidity'] < 60)
].copy()

finest_cities

,City Name,Latitude,Longitude,Daily High,Humidity,Cloudiness,Wind Speed,Country,Date
12,Tiznit Province,29.58,-9.50,71.13,47,0,1.74,MA,2020-07-10
142,Antsohihy,-14.88,47.99,73.06,54,0,6.87,MG,2020-07-10
189,Baker City,44.77,-117.83,66.20,52,1,5.82,US,2020-07-10
296,Tooele,40.53,-112.30,73.00,34,1,6.93,US,2020-07-10
302,Mountain Home,43.13,-115.69,71.60,26,1,2.57,US,2020-07-10
343,Alta Floresta,-9.88,-56.09,67.73,58,0,2.04,BR,2020-07-10
364,Susanville,40.42,-120.65,73.40,13,1,9.17,US,2020-07-10
416,Makīnsk,52.63,70.42,73.17,50,0,8.97,KZ,2020-07-10
456,Ziarat,30.38,67.73,73.83,24,0,4.14,PK,2020-07-10
509,Empangeni,-28.76,31.89,69.15,43,0,9.82,ZA,2020-07-10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = 'Store'

finest_cities['Hotel Name'] = ''

locations = finest_cities[['Latitude','Longitude']]

,City Name,Latitude,Longitude,Daily High,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
12,Tiznit Province,29.58,-9.50,71.13,47,0,1.74,MA,2020-07-10,
142,Antsohihy,-14.88,47.99,73.06,54,0,6.87,MG,2020-07-10,
189,Baker City,44.77,-117.83,66.20,52,1,5.82,US,2020-07-10,
296,Tooele,40.53,-112.30,73.00,34,1,6.93,US,2020-07-10,
302,Mountain Home,43.13,-115.69,71.60,26,1,2.57,US,2020-07-10,
343,Alta Floresta,-9.88,-56.09,67.73,58,0,2.04,BR,2020-07-10,
364,Susanville,40.42,-120.65,73.40,13,1,9.17,US,2020-07-10,
416,Makīnsk,52.63,70.42,73.17,50,0,8.97,KZ,2020-07-10,
456,Ziarat,30.38,67.73,73.83,24,0,4.14,PK,2020-07-10,
509,Empangeni,-28.76,31.89,69.15,43,0,9.82,ZA,2020-07-10,


In [ ]:
# NOTE: Do *** change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in finest_cities.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
